In [80]:
import pandas as pd
import numpy as np
from datetime import timedelta


In [81]:
df = pd.read_csv("Dataset.csv")

df["Timestamp"] = pd.to_datetime(
    df["Time interval (CET/CEST)"].str.split(" - ").str[1],
    dayfirst=True
)

df["Price"] = df["Price"].astype(float)
df = df.sort_values("Timestamp").reset_index(drop=True)


In [82]:
df["hour"] = df["Timestamp"].dt.hour
df["weekday"] = df["Timestamp"].dt.weekday
df["month"] = df["Timestamp"].dt.month

# Medii istorice pe oră
hourly_mean = df.groupby("hour")["Price"].mean()

# Medii istorice pe zi a săptămânii
weekday_mean = df.groupby("weekday")["Price"].mean()

# Volatilitatea recentă (std pe 48 intervale)
df["rolling_std"] = df["Price"].rolling(48).std().fillna(df["Price"].std())

# Trend pe ultima săptămână (7 zile = 7*96 intervale)
df["rolling_mean_7d"] = df["Price"].rolling(7*96).mean().fillna(df["Price"].mean())


In [83]:
P1  = df["Price"].quantile(0.01)
P10 = df["Price"].quantile(0.10)
P90 = df["Price"].quantile(0.90)
P99 = df["Price"].quantile(0.99)

print(P1, P10, P90, P99)


-5264.63026104578 -924.2874561575507 2635.559715629548 5648.185040987071


In [84]:
sample = pd.read_csv("sample_submission.csv")

future_dates = pd.to_datetime(
    sample["Time interval (CET/CEST)"].str.split(" - ").str[1],
    dayfirst=True
)

df_future = pd.DataFrame({
    "Time interval (CET/CEST)": sample["Time interval (CET/CEST)"],
    "Timestamp": future_dates
})

df_future["hour"] = df_future["Timestamp"].dt.hour
df_future["weekday"] = df_future["Timestamp"].dt.weekday


In [100]:
forecast_prices = []

last_week_mean = df["rolling_mean_7d"].iloc[-1]
last_vol = df["rolling_std"].iloc[-1]

for _, row in df_future.iterrows():

    hour = row["hour"]
    weekday = row["weekday"]

    # bază mai variabilă
    base = (
        1.5 * hourly_mean[hour] +
        1.0 * weekday_mean[weekday] -
        0.5 * last_week_mean
    )

    # noise EXTREM pentru a produce distribuție mare
    noise = np.random.normal(0, last_vol * 1.5)

    # probabilitate să apară spike-uri pozitive/negative
    if np.random.rand() < 0.05:  # 5% din timp spike pozitiv
        spike = np.random.uniform(2000, 6000)
        forecast_prices.append(base + noise + spike)
        continue

    if np.random.rand() < 0.05:  # 5% din timp spike negativ
        spike = -np.random.uniform(2000, 6000)
        forecast_prices.append(base + noise + spike)
        continue

    forecast_prices.append(base + noise)

df_future["Price"] = forecast_prices


In [101]:
SOC_MIN = 0.0
SOC_MAX = 10.0
SOC_START_DAY = 5.0
MIN_TR = 0.1

def choose_action(price, hour, soc, p1, p10, p90, p99):

    # Final de zi: vinde tot ce ai
    if hour >= 21:
        if soc > 0:
            return -min(soc, 2.0)
        else:
            return MIN_TR

    if price < p1:
        return 2.0
    if price < p10:
        return 1.0

    if price <= p90:
        if soc < 5.0:
            return MIN_TR
        elif soc > 5.0:
            return -MIN_TR
        else:
            return MIN_TR if hour < 12 else -MIN_TR

    if price < p99:
        return -1.0

    return -2.0


In [102]:
def run_strategy(dfp, p1, p10, p90, p99):
    
    actions = []
    soc_history = []
    soc = SOC_START_DAY
    prev_day = None

    for _, row in dfp.iterrows():
        
        ts = row["Timestamp"]
        price = row["Price"]
        
        hour = ts.hour
        day = ts.date()

        # Reset zilnic
        if prev_day is None or day != prev_day:
            soc = SOC_START_DAY
            prev_day = day

        raw_action = choose_action(price, hour, soc, p1, p10, p90, p99)

        cap_up = SOC_MAX - soc
        cap_down = soc - SOC_MIN

        action = raw_action

        # Prevenim încărcarea peste limită
        if action > 0 and cap_up < MIN_TR:
            action = -MIN_TR if cap_down >= MIN_TR else 0.0

        # Prevenim descărcarea sub limită
        if action < 0 and cap_down < MIN_TR:
            action = MIN_TR if cap_up >= MIN_TR else 0.0

        if 0 < abs(action) < MIN_TR:
            action = MIN_TR if action > 0 else -MIN_TR

        new_soc = soc + action

        new_soc = max(min(new_soc, SOC_MAX), SOC_MIN)

        soc = round(new_soc, 4)

        actions.append(action)
        soc_history.append(soc)

    return actions, soc_history


In [103]:
actions, soc_hist = run_strategy(df_future, P1, P10, P90, P99)


In [104]:
submission = pd.DataFrame({
    "Time interval (CET/CEST)": df_future["Time interval (CET/CEST)"],
    "Position": actions
})

submission.to_csv("submission_heuristic.csv", index=False)
submission.head(20)


,Time interval (CET/CEST),Position
0,26.10.2025 00:00 - 26.10.2025 00:15,0.1
1,26.10.2025 00:15 - 26.10.2025 00:30,-0.1
2,26.10.2025 00:30 - 26.10.2025 00:45,2.0
3,26.10.2025 00:45 - 26.10.2025 01:00,-0.1
4,26.10.2025 01:00 - 26.10.2025 01:15,-0.1
5,26.10.2025 01:15 - 26.10.2025 01:30,-0.1
6,26.10.2025 01:30 - 26.10.2025 01:45,-0.1
7,26.10.2025 01:45 - 26.10.2025 02:00,-0.1
8,26.10.2025 02:00 - 26.10.2025 02:15,-0.1
9,26.10.2025 02:15 - 26.10.2025 02:30,1.0
